In [65]:
import pandas as pd
import re
import subprocess
import uuid
import csv
import json
import hashlib

## Read the Config

In [61]:
with open('config/exportMetadata.v2.json', 'r') as infile:
    exportMetadata = json.load(infile)


{'name': 'tf_Example-1701471595480',
 'sourceOrgId': '2lh8CM1r57OkWgGofQVMIk',
 'sourceOrgName': 'Jon Bowring Suborg',
 'exportedObjects': [{'objectGuid': 'aJDWwb7VFSmbzZ86glzi20',
   'objectName': 'tf_Example',
   'objectType': 'TASKFLOW',
   'path': '/Explore/Default',
   'metadata': {'modelVersion': {'major': 1, 'minor': 0},
    'repoInfo': {'repoHandle': '3RKWa-gt-522578-2023-12-01T22:58:53.881Z::tf.xml'},
    'objectRefs': ['1jY0fuy0iEUhkrHVLx78WK'],
    'contextAttributes': None,
    'additionalInfo': {'description': '',
     'contentType': 'application/json; charset=utf-8',
     'documentState': 'VALID'}}},
  {'objectGuid': '92TQndHw9hvjJrQafpLX2R',
   'objectName': 'Default',
   'objectType': 'Project',
   'path': '/Explore',
   'metadata': {'modelVersion': {'major': 0, 'minor': 0},
    'repoInfo': None,
    'objectRefs': [],
    'contextAttributes': [{'name': 'IS_DEF_LOC', 'value': 'true'}],
    'additionalInfo': {'description': 'Auto-generated Default Project',
     'contentT

## Read the plans

In [39]:
# Read the execution plans
dfPlans = pd.read_csv('in/plans.csv', dtype='str', encoding='utf-8', na_filter=False)

In [40]:
# Get a list of unique execution plans
plans = dfPlans['plan_name'].unique()
plans

array(['Echo Employee Snapshot Oracle R12.1.3'], dtype=object)

## Create the Import Packages

In [68]:
# For each unique plan
for plan in plans:
    
    # Create IDs and a clean version of the name for the new taskflow
    tName = re.sub(r'[^\w\d]+', '_', plan)
    projectID = str(uuid.uuid4())
    taskflowID = str(uuid.uuid4())
    mapID = '1jY0fuy0iEUhkrHVLx78WK' # TODO Update to dynamically get the correct mapping

    # Create the ContentsofExportPackage File
    content = {'objectPath': ['/Explore/Default','/Explore'], 'objectName': [tName, 'Default'], 'objectType': ['TASKFLOW', 'Project'], 'id': [taskflowID, projectID]}
    contentDf = pd.DataFrame(data=content)
    contentDf.to_csv(f'out/ContentsofExportPackage_{tName}.csv', index=False, quoting=csv.QUOTE_NONE)

    # Create the exportMetadata file
    exportMetadata['name'] = tName
    taskflowObj = list(filter(lambda x: x['objectType'] == 'TASKFLOW', exportMetadata['exportedObjects']))[0]
    taskflowObj['objectGuid'] = taskflowID
    taskflowObj['objectName'] = tName
    taskflowObj['objectRefs'] = ['1jY0fuy0iEUhkrHVLx78WK'] # TODO Update to dynamically get the correct mapping
    projectObj = list(filter(lambda x: x['objectType'] == 'Project', exportMetadata['exportedObjects']))[0]
    projectObj['objectGuid'] = projectID
    mappingObj = list(filter(lambda x: x['objectType'] == 'MTT', exportMetadata['exportedObjects']))[0]
    mappingObj['objectGuid'] = '1jY0fuy0iEUhkrHVLx78WK' # TODO Update to dynamically get the correct mapping
    mappingObj['objectName'] = 'mt_Example' # TODO Update to dynamically get the correct mapping
    exportMetadata['exportedObjects'] = [taskflowObj, projectObj, mappingObj]
    with open('out/exportMetadata.v2.json', 'w') as outfile:
        outfile.write(json.dumps(exportMetadata))

    # Run the XQuery conversion
    console = subprocess.run(["java", "-Xmx14000M", "-cp", "../saxon-he-10.5.jar", "net.sf.saxon.Query", "-q:convert.xq", f"-o:out/Explore/Default/{tName}.TASKFLOW.xml", f"tname={tName}", f"tflowid={taskflowID}"], capture_output=True)
    
    # Create the exportPackage.chksum file
    with open('out/Explore/Default.Project.json', 'rb') as infile:
        infileBytes = infile.read()
        defaultProjectJsonHash = hashlib.sha256(infileBytes).hexdigest().upper()

    with open('out/exportMetadata.v2.json', 'rb') as infile:
        infileBytes = infile.read()
        exportMetadataHash = hashlib.sha256(infileBytes).hexdigest().upper()

    with open(f'out/Explore/Default/{tName}.TASKFLOW.xml', 'rb') as infile:
        infileBytes = infile.read()
        taskflowHash = hashlib.sha256(infileBytes).hexdigest().upper()
    
    lines = ['#\n', '#Fri Dec 01 22:59:59 UTC 2023\n', f'Explore/Default.Project.json={defaultProjectJsonHash}\n', f'exportMetadata.v2.json={exportMetadataHash}\n', f'Explore/Default/tf_Example.TASKFLOW.xml={taskflowHash}\n']
    with open('out/exportPackage.chksum', 'w') as outfile:
        outfile.writelines(lines)